In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
rawdata = pd.read_excel(f"{os.path.dirname(os.getcwd())}\\raw_data\\all_matches_6ovr_data.xlsx",sheet_name="all_matches_6ovr_data (8 Teams)")

In [3]:
df = rawdata.copy()
df['start_date'] = pd.to_datetime(df['start_date'])
df = df.sort_values(by='start_date')

In [8]:
df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'venue_Updated', 'innings',
       'ball', 'Over', 'batting_team', 'Standardised_batting_team',
       'bowling_team', 'Standardised_bowling_team', 'striker', 'non_striker',
       'bowler', 'runs_off_bat', 'extras', 'Overall_Runs', 'wides', 'noballs',
       'byes', 'legbyes', 'penalty', 'wicket_type', 'player_dismissed',
       'other_wicket_type', 'other_player_dismissed', 'Total_runs', 'PP_Score',
       'is_wicket', 'Total_wickets', 'PP_wickets'],
      dtype='object')

In [5]:
# * running total of overall runs
df['Total_runs'] = df.groupby(['match_id','innings'])['Overall_Runs'].apply(lambda x: x.cumsum())
df['PP_Score'] = df.groupby(['match_id','innings'])['Overall_Runs'].transform('sum')

In [6]:
# * Wickets
df['is_wicket'] = df['wicket_type'].apply(lambda x: 0 if pd.isna(x) else 1)
# * running total of no of wickets
df['Total_wickets'] = df.groupby(['match_id','innings'])['is_wicket'].apply(lambda x: x.cumsum())
df['PP_wickets'] = df.groupby(['match_id','innings'])['is_wicket'].transform('sum')

In [10]:
df.to_csv(f"{os.path.dirname(os.getcwd())}\\raw_data\\data_by_overs.csv")

### Aggregating data to innings level

In [24]:
data_ings_level = df[['match_id','season', 'start_date', 'venue_Updated', 'innings','Standardised_batting_team', 'Standardised_bowling_team','PP_Score','PP_wickets']]

In [25]:
# drop duplicates to have one record for each innings in a match
data_ings_level = data_ings_level.drop_duplicates()

In [57]:
# * first innings data
first_ings = data_ings_level[data_ings_level['innings'] == 1]
first_ings['avgscore_TD_by_venue'] = first_ings.groupby(['venue_Updated'])['PP_Score'].apply(lambda x: x.cumsum()/range(1,len(x)+1))
first_ings['avgwkts_TD_by_venue'] = first_ings.groupby(['venue_Updated'])['PP_wickets'].apply(lambda x: x.cumsum()/range(1,len(x)+1))

# * second innings data
second_ings = data_ings_level[data_ings_level['innings'] == 2]
second_ings['avgscore_TD_by_venue'] = second_ings.groupby(['venue_Updated'])['PP_Score'].apply(lambda x: x.cumsum()/range(1,len(x)+1))
second_ings['avgwkts_TD_by_venue'] = second_ings.groupby(['venue_Updated'])['PP_wickets'].apply(lambda x: x.cumsum()/range(1,len(x)+1))

<ipython-input-57-23deaede02ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_ings['avgscore_TD_by_venue'] = first_ings.groupby(['venue_Updated'])['PP_Score'].apply(lambda x: x.cumsum()/range(1,len(x)+1))
<ipython-input-57-23deaede02ff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_ings['avgwkts_TD_by_venue'] = first_ings.groupby(['venue_Updated'])['PP_wickets'].apply(lambda x: x.cumsum()/range(1,len(x)+1))


<ipython-input-58-cb645fffa005>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_ings['avgscore_TD_by_venue'] = second_ings.groupby(['venue_Updated'])['PP_Score'].apply(lambda x: x.cumsum()/range(1,len(x)+1))
<ipython-input-58-cb645fffa005>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_ings['avgwkts_TD_by_venue'] = second_ings.groupby(['venue_Updated'])['PP_wickets'].apply(lambda x: x.cumsum()/range(1,len(x)+1))


In [75]:
# * calculate average for each batting team
list_df=[]
for ing in [first_ings,second_ings]:
    for teams in ing['Standardised_batting_team'].unique():
        team_data = ing[ing['Standardised_batting_team'] == teams]
        team_data['avgscore_TD_by_bat_team'] = team_data.groupby(['Standardised_batting_team'])['PP_Score'].apply(lambda x: x.cumsum()/range(1,len(x)+1))
        team_data['avgwkts_TD_by__bat_team'] = team_data.groupby(['Standardised_batting_team'])['PP_wickets'].apply(lambda x: x.cumsum()/range(1,len(x)+1))
        list_df.append(team_data)

data_ings_level = pd.concat(list_df,axis=0)

<ipython-input-75-540b22efb8d7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data['avgscore_TD_by_bat_team'] = team_data.groupby(['Standardised_batting_team'])['PP_Score'].apply(lambda x: x.cumsum()/range(1,len(x)+1))
<ipython-input-75-540b22efb8d7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data['avgwkts_TD_by__bat_team'] = team_data.groupby(['Standardised_batting_team'])['PP_wickets'].apply(lambda x: x.cumsum()/range(1,len(x)+1))


In [78]:
data_ings_level

,match_id,season,start_date,venue_Updated,innings,Standardised_batting_team,Standardised_bowling_team,PP_Score,PP_wickets,avgscore_TD_by_venue,avgwkts_TD_by_venue,avgscore_TD_by_bat_team,avgwkts_TD_by__bat_team
1280,336000,2007/08,2008-01-05,Sawai Mansingh Stadium,1,Rajasthan Royals,Kolkata Knight Riders,45,1,45.000000,1.000000,45.000000,1.000000
171,335984,2007/08,2008-04-19,Arun Jaitley Stadium,1,Rajasthan Royals,Delhi Capitals,40,2,40.000000,2.000000,42.500000,1.500000
2894,336023,2007/08,2008-05-17,Sawai Mansingh Stadium,1,Rajasthan Royals,Royal Challengers Bangalore,51,0,47.000000,1.500000,45.333333,1.000000
3665,336033,2007/08,2008-05-24,"MA Chidambaram Stadium, Chepauk, Chennai",1,Rajasthan Royals,Chennai Super Kings,67,0,50.200000,1.600000,50.750000,0.750000
4181,336038,2007/08,2008-05-30,"Wankhede Stadium, Mumbai",1,Rajasthan Royals,Delhi Capitals,58,0,40.600000,1.800000,52.200000,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48170,1216496,2020/21,2020-09-22,Sharjah Cricket Stadium,2,Chennai Super Kings,Rajasthan Royals,53,0,47.818182,0.818182,46.443038,1.303797
48318,1216539,2020/21,2020-09-25,Dubai International Cricket Stadium,2,Chennai Super Kings,Delhi Capitals,34,2,40.647059,2.235294,46.287500,1.312500
49748,1216525,2020/21,2020-10-10,Dubai International Cricket Stadium,2,Chennai Super Kings,Royal Challengers Bangalore,26,2,39.550000,2.200000,46.037037,1.320988
51227,1216544,2020/21,2020-10-25,Dubai International Cricket Stadium,2,Chennai Super Kings,Royal Challengers Bangalore,48,1,43.275862,1.965517,46.060976,1.317073


In [79]:
data_ings_level.to_csv(f"{os.path.dirname(os.getcwd())}\\raw_data\\data_ings_level.csv")